In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
pip install pyspark

In [ ]:
pip install sgt

In [ ]:
pip install pandarallel

In [6]:
import sgt
sgt.__version__
from sgt import SGT

In [7]:
import numpy as np
import pandas as pd
from itertools import chain
from itertools import product as iterproduct
import warnings
from pandarallel import pandarallel  # required library for multiprocessing

import pickle

########
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics
import time

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(7) # fix random seed for reproducibility



In [8]:
train= pd.read_csv('/gdrive/My Drive/Capstone/CNN-Data.csv',sep=',',header=0,skipinitialspace=True) # train data set

In [ ]:
train.head()

In [ ]:
seq= train['aminoAcid'].map(list)
seq=seq.to_dict()
seq = list(map(list, seq.items()))
corpus = pd.DataFrame(seq, columns= ['id','sequence'])
corpus

In [12]:
alphabets = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 
             'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 
             'W', 'X', 'Y', 'U', 'O']  # List of amino acids

sgt_ = SGT(alphabets=alphabets, lengthsensitive=True, kappa=1, flatten=True,mode='multiprocessing')
sgtembedding_df = sgt_.fit_transform(corpus)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [14]:
sgtembedding_df = sgtembedding_df.set_index('id')
sgtembedding_df

,"(A, A)","(A, C)","(A, D)","(A, E)","(A, F)","(A, G)","(A, H)","(A, I)","(A, K)","(A, L)","(A, M)","(A, N)","(A, P)","(A, Q)","(A, R)","(A, S)","(A, T)","(A, V)","(A, W)","(A, X)","(A, Y)","(A, U)","(A, O)","(C, A)","(C, C)","(C, D)","(C, E)","(C, F)","(C, G)","(C, H)","(C, I)","(C, K)","(C, L)","(C, M)","(C, N)","(C, P)","(C, Q)","(C, R)","(C, S)","(C, T)",...,"(U, H)","(U, I)","(U, K)","(U, L)","(U, M)","(U, N)","(U, P)","(U, Q)","(U, R)","(U, S)","(U, T)","(U, V)","(U, W)","(U, X)","(U, Y)","(U, U)","(U, O)","(O, A)","(O, C)","(O, D)","(O, E)","(O, F)","(O, G)","(O, H)","(O, I)","(O, K)","(O, L)","(O, M)","(O, N)","(O, P)","(O, Q)","(O, R)","(O, S)","(O, T)","(O, V)","(O, W)","(O, X)","(O, Y)","(O, U)","(O, O)"
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.000000,1.296254,0.647928,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.047166,0.000000,0.0,0.0,0.128209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,0.0,0.0,0.0,0.000000,0.003019,2.264466,0.000000,0.00000,0.164841,0.000000,0.0,0.000000,0.000000,0.022309,0.0,0.000000,0.060642,0.448084,0.0,0.0,0.008207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.000000,0.001234,0.000000,0.678354,0.00000,0.000000,3.678794,0.0,0.183156,0.024788,0.038249,0.0,0.497871,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.060642,0.003019,0.000000,0.000000,0.00000,0.000000,1.879499,0.0,0.000000,1.218018,0.022309,0.0,0.000000,0.000000,0.164841,0.0,0.0,0.008207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.000000,0.000080,0.011854,0.000000,0.00059,0.000000,0.647232,0.0,0.004361,3.270896,0.238103,0.0,0.032224,0.001604,0.000000,0.0,0.0,0.043905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225688.0,0.0,0.0,0.0,0.000000,0.114545,0.000000,0.000000,0.00000,0.000000,0.846380,0.0,6.253950,0.000000,0.000000,0.0,0.000000,0.311366,2.300700,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225689.0,0.0,0.0,0.0,0.000000,0.114545,0.000000,0.000000,0.00000,0.000000,0.846380,0.0,6.253950,0.000000,0.000000,0.0,0.000000,0.311366,2.300700,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225690.0,0.0,0.0,0.0,0.000000,0.094331,0.000000,0.000000,0.00000,0.000000,

In [15]:
pca = PCA(n_components=60)
pca.fit(sgtembedding_df)
X = pca.transform(sgtembedding_df)
print(np.sum(pca.explained_variance_ratio_))
PCA_components = pd.DataFrame(X)

0.7174676630955962


In [16]:
max_seq_length = np.max(train['aminoAcid'])

In [17]:
y = train['aminoAcid']
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

In [19]:
kfold = 2
random_state = 11
#X= X[1:2]
test_F1 = np.zeros(kfold)
time_k = np.zeros(kfold)

epochs = 50
batch_size = 15
skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=random_state)

k = 0

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    embedding_vecor_length = 32
    top_words=50
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=max_seq_length))
    model.add(LSTM(32))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    
    start_time = time.time()
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    end_time=time.time()
    time_k[k]=end_time-start_time

    y_pred = model.predict_proba(X_test).round().astype(int)
    y_train_pred=model.predict_proba(X_train).round().astype(int)
    test_F1[k]=sklearn.metrics.f1_score(y_test, y_pred)
    k+=1

ValueError: ignored